In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from scipy import signal
from scipy.signal import windows
import scipy.io
from glob import glob
import dask.dataframe as dd
import dask.array as da
from dask import delayed
from distutils.dir_util import copy_tree
import shutil

In [3]:
@delayed
def dot_p_b(x,y,n_rows,n_features,no_norm):
    #x -> t_T
    #y -> X_b

    if y.shape!=(n_rows,n_features):
        y = y.transpose()
    pp_1 = (x.transpose().dot(y)) #1,8193
    pp_2 = (x.transpose().dot(x))
    pp_3 = pp_1/pp_2
    if no_norm == False:
        pp_4 = (pp_3)/np.linalg.norm(pp_3)

        return pp_4

    else:

        return pp_3


@delayed
def dot_t_b(x,y,n_rows,n_features):
    #x -> p_b
    #y -> X_b
    #print(x)
    #print(y)
    x = x.reshape(-1,1)

    if y.shape!=(n_features,n_rows):
        y = y.transpose()
    pp_1 = (x.transpose().dot(y))
    pp_2 = pp_1/np.sqrt(n_features)
    #print(pp_2.shape)
    if pp_2.shape!=(n_rows,1):
        pp_2 = pp_2.transpose()

    return pp_2

@delayed
def compute_var_explained(x,y):
    #x -> residuals
    #y -> blocks

    var_exp = abs(1 - ((np.trace(x.transpose().dot(x))/(np.trace(y.transpose().dot(y))))))*100
    return var_exp, 

def read_hdfs(root,read_keys) -> dd:

    file_path = "../Bridge_Data/scaled_201901_*.h5"
    files = glob(file_path)
    print(files)

    padded_files = sorted(files, key = lambda x: (int(x.split('_')[-7]), int(x.split('_')[-5]), int(x.split('_')[-3])))
    df = dd.read_hdf(padded_files,key='data',lock=True,sorted_index=True)
    return df, padded_files


In [4]:
class nipals_methods:
    def __init__(self,root):
        self.root = root
        self.t_T = pd.read_hdf(self.root+'scaled_201901_day_1_sensor_1_parallel_padded.h5',key='data').transpose()
        #print(self.t_T.shape)
        self.n_features = self.t_T.shape[1]
        self.n_rows = self.t_T.shape[0]
        self.t_T = np.asarray([-0.4109466, -0.40807255, 2.04122889, -0.40160283, -0.40939843, -0.41120847]).reshape(-1,1)#([-0.40824829, -0.16738194, -0.82674259,  0.06009628, -0.19901919,-0.28034463]).reshape(-1,1)
        #self.folders = glob(self.root+'/sensor_data_*')
        #self.folders.sort()

    def pad_missing_values(self,read_keys):
        self.block_files = []
        for folder in self.folders:
            key = folder.split('_',folder.count('_'))[-1]
            if key in read_keys:
                self.block_files.extend(glob.glob(self.root+'/sensor_data_'+key+'/scaled_*_parallel.h5'))
        for file in self.block_files:
            df = pd.read_hdf(file,key='data',mode='r')
            if df.shape[1] != self.n_rows:
                cols = df.shape[1]
                for i in range(cols,self.n_rows):
                    df.loc[:,i] = 0
            df.to_hdf(file.replace('.h5','_padded.h5'),key='data',format='table')

    def compute_block_loadings(self,no_norm=False):
        p_b = []
        df_map = dd.map_partitions(dot_p_b,self.t_T,self.df,self.n_rows,self.n_features,no_norm)
        delayed_fncs = df_map.compute()
        print(np.linalg.norm(delayed_fncs[1].compute()))
        for i in range(len(delayed_fncs)):
          p_b.append(delayed_fncs[i].compute())

        self.p_b = p_b

    def compute_block_scores(self):
        self.t_b = []
        for i in range(len(self.p_b[0])):
          df_map = dd.map_partitions(dot_t_b,self.p_b[i][0],self.df,self.n_rows,self.n_features,align_dataframes=True)
          delayed_fncs = df_map.compute()
          #print(delayed_fncs[1].compute())
          for j in range(len(delayed_fncs)):
            self.t_b.append(delayed_fncs[j].compute())
        #print(self.t_b.shape)
        self.T = np.reshape(self.t_b,(self.n_rows,len(self.t_b)))
        print("T shape")
        print(self.T.shape)

    def compute_super_weight(self):
        pp_1 = self.T.transpose().dot(self.t_T)
        pp_2 = (self.t_T.transpose().dot(self.t_T))
        pp_3 = pp_1/pp_2
        self.w_T = pp_3/np.linalg.norm(pp_3)

    def update_super_score(self):
        self.t_T_new = self.T.dot(self.w_T)
        t_T = self.t_T_new
        return t_T

    def deflate(self):
        E = []
        start = 0
        chunk = self.n_features
        self.compute_block_loadings(no_norm=True)
        self.p_b = np.reshape(self.p_b,(self.n_partitions,self.n_features))
        df = self.df.compute()
        for i in range(self.p_b.shape[0]):
            p_b = self.p_b[i].reshape(-1,1)
            pp_1 = self.t_T.dot(p_b.transpose())
            E.append(df[start:start+chunk].transpose() - pp_1)
            start += self.n_features
        E = np.reshape(E,(self.n_partitions,self.n_features,self.n_rows))
        E = np.vstack(E)
        df = pd.DataFrame(E,columns=pd.Index([0,1,2,3,4,5],dtype='int64'))
        df = dd.from_pandas(df,npartitions=self.n_partitions)
        self.df = df
        return df

    def fit(self,X,tolerance,max_iter,n_components,read_ckp=None,write_ckp=None):
        X = X.reset_index(drop=True)

        self.df = X

        self.n_partitions = self.df.npartitions

        model = {'t_T_scores':np.empty((0,self.n_rows)),'p_b':np.empty((0,self.n_partitions,self.n_features)),'t_b':np.empty((0,self.n_partitions,self.n_rows)),'w_T':np.empty((0,self.n_partitions)),'residuals':np.empty((0,self.n_features,self.n_rows)),'cum_var_exp':np.empty((0,self.n_partitions))}
        ckp_components = 0
        total_components = n_components

        if not (read_ckp == None):

            model['cum_var_exp'] = pd.read_hdf(glob.glob(self.root+'/cum_var_exp_ckp.h5')[0],key='data').values
            ckp_components = model['cum_var_exp'].shape[0]
            ckp_components = int(ckp_components)
            total_components = n_components + ckp_components
            total_components = int(total_components)

            if ckp_components+n_components > self.n_features:
                print('Error: Fitting too many components.')
                return

            model['t_T_scores'] = pd.read_hdf(glob.glob(self.root+'/t_T_score_ckp.h5')[0],key='data').values

            model['p_b'] = pd.read_hdf(glob.glob(self.root+'/p_b_stack_ckp.h5')[0],key='data').values
            model['p_b'] = model['p_b'].reshape(ckp_components,self.n_partitions,self.n_features)

            model['t_b'] = pd.read_hdf(glob.glob(self.root+'/t_b_stack_ckp.h5')[0],key='data').values
            model['t_b'] = model['t_b'].reshape(ckp_components,self.n_partitions,self.n_rows)

            model['w_T'] = pd.read_hdf(glob.glob(self.root+'/w_T_ckp.h5')[0],key='data').values

            residuals = pd.read_hdf(glob.glob(self.root+'/residuals_stack_ckp.h5')[0],key='data')
            residuals = residuals.reset_index(drop=True)
            model['residuals'] = dd.from_pandas(residuals,self.n_partitions)
            model['residuals'] = model['residuals'].reset_index(drop=True)

            self.t_T = np.array(model['t_T_scores'][-1]).reshape(-1,1)

            self.df = model['residuals']

            if n_components == 0: return model


        for i in range(ckp_components,total_components):

            print('Fitting component ',i+1)
            #print(self.t_T)

            for j in range(max_iter):

                print('Current Iteration: ',j)
                self.compute_block_loadings()
                self.compute_block_scores()
                self.compute_super_weight()
                #print("super")
                #print(self.T)
                t_T = self.update_super_score()
                eps = abs(t_T - self.t_T)
                if np.all(eps<tolerance):

                    print('Component converged early. Iteration: ',j)
                    self.t_T = t_T
                    break

                self.t_T = t_T

            self.p_b = np.reshape(self.p_b,(1,self.n_partitions,self.n_features))
            model['p_b'] = np.concatenate((model['p_b'],self.p_b),axis=0)
            model['t_T_scores'] = np.append(model['t_T_scores'],self.t_T.transpose(),axis=0)
            model['t_b'] = np.append(model['t_b'],np.reshape(self.t_b,(1,self.n_partitions,self.n_rows)),axis=0)
            model['w_T'] = np.append(model['w_T'],self.w_T.transpose(),axis=0)

            self.residuals = self.deflate()
            self.residuals = self.residuals.reset_index(drop=True)

            test = np.trace(X)

            df_map = dd.map_partitions(compute_var_explained,self.residuals,X)
            delayed_fncs = df_map.compute()
            delayed_vals = []

            for func in range(len(delayed_fncs)):

                delayed_vals.append(delayed_fncs[func].compute())

            delayed_vals = np.reshape(delayed_vals,(1,self.n_partitions))
            model['cum_var_exp'] = np.append(model['cum_var_exp'],delayed_vals,axis=0)

        residuals = self.residuals.compute()
        residuals = residuals.values
        residuals = residuals.reshape(self.n_partitions,self.n_features,self.n_rows)
        model['residuals'] = residuals

        if not (write_ckp == None):
            self.write_checkpoint(model,total_components)
        print(type(model))
        return model

    def write_checkpoint(self,model,n_components):

        cum_var_exp = np.reshape(model['cum_var_exp'],(n_components,self.n_partitions))
        cum_var_exp = pd.DataFrame(cum_var_exp)
        cum_var_exp.to_hdf(root+'cum_var_exp_ckp.h5',key='data',format='fixed')

        residuals = np.reshape(model['residuals'],(self.n_partitions,self.n_features,self.n_rows))
        residuals = np.vstack(model['residuals'])
        residuals = pd.DataFrame(residuals)
        residuals.to_hdf(root+'residuals_stack_ckp.h5',key='data',format='table')

        p_b_stack = np.reshape(model['p_b'],(n_components,self.n_partitions,self.n_features))
        p_b_stack = np.vstack(p_b_stack)
        p_b_stack = pd.DataFrame(p_b_stack)
        p_b_stack.to_hdf(root+'p_b_stack_ckp.h5',key='data',format='fixed')

        t_T_score = np.reshape(model['t_T_scores'],(n_components,self.n_rows))
        t_T_score = pd.DataFrame(t_T_score)
        t_T_score.to_hdf(root+'t_T_score_ckp.h5',key='data',format='fixed')

        t_b_stack = np.reshape(model['t_b'],(n_components,self.n_partitions,self.n_rows))
        t_b_stack = np.vstack(t_b_stack)
        t_b_stack = pd.DataFrame(t_b_stack)
        t_b_stack.to_hdf(root+'t_b_stack_ckp.h5',key='data',format='fixed')

        w_T = np.reshape(model['w_T'],(n_components,self.n_partitions))
        w_T = pd.DataFrame(w_T)
        w_T.to_hdf(root+'w_T_ckp.h5',key='data',format='fixed')


In [5]:
root = 'E:/Projects/Bridge_Data/'

In [ ]:
#original
blocks, files = read_hdfs(root,['201811'])
#,'201811','201812','201901','201911','201912'
#blocks = read_hdfs(root,['201811','201812','201901','201902','201910','201911','201912','202001'])

In [8]:
#meow = pd.DataFrame([-0.40824829, -0.16738194, -0.82674259,  0.06009628, -0.19901919,-0.28034463]).values

In [7]:
test = nipals_methods(root)

In [15]:
test_block = (blocks.get_partition(1).values).compute()

In [16]:
np.trace(test_block.T @ test_block)

40964.999999999985

In [9]:
model = test.fit(X=blocks,tolerance=0.01,max_iter=5,n_components=10,read_ckp=None,write_ckp=False)

Fitting component  1
Current Iteration:  0
1.0
T shape
(6, 186)
Current Iteration:  1
1.0
T shape
(6, 186)
Current Iteration:  2
1.0000000000000002
T shape
(6, 186)
Current Iteration:  3


KeyboardInterrupt: 

In [ ]:
meower = pd.DataFrame(model['cum_var_exp'])
himeow = pd.DataFrame(meower.iloc[0]).transpose().values
meowermeow = pd.DataFrame(meower.diff())
meowermeow.loc[0] = himeow

In [ ]:
model['p_b'].shape

In [ ]:
meower

,0,1,2,3,4,5,6,7,8,9,...,176,177,178,179,180,181,182,183,184,185
0,0.196734,0.176503,0.161266,0.202313,0.190423,0.165380,0.108184,0.215724,0.260643,0.172995,...,0.305486,0.327785,0.327310,0.298450,0.324313,0.297171,0.281237,0.258994,0.279247,0.266429
1,52.000804,54.937735,61.687377,56.305102,58.700754,59.213271,65.563600,66.229601,66.757297,65.935608,...,66.168693,64.275857,65.982339,65.761028,65.002893,65.091513,66.063531,64.121377,65.548320,65.385335
2,68.693309,72.365811,79.900612,73.634129,76.451273,76.943880,84.752932,85.860328,86.439462,85.430024,...,85.244460,82.845602,84.991646,84.764487,84.220929,84.055758,85.092595,82.844576,84.656481,84.366907
3,75.363649,78.895220,85.348298,79.834431,82.400146,82.820002,89.785119,90.918085,91.354940,90.471377,...,90.356929,88.244589,89.971956,89.813887,89.408252,89.189732,90.096273,88.052841,89.736807,89.394582
4,79.784130,82.891797,88.296453,83.716379,85.947966,86.259472,92.180101,93.192004,93.595380,92.790076,...,92.738213,90.938071,92.334795,92.259503,91.782192,91.661129,92.405538,90.625249,92.073670,91.840617
5,83.192011,85.860568,90.325336,86.514209,88.349305,88.567400,93.584355,94.458460,94.749734,94.095940,...,94.138242,92.618925,93.765983,93.715785,93.234161,93.120322,93.842270,92.326275,93.553588,93.348242
6,86.093505,88.217478,91.988524,88.774929,90.250413,90.482843,94.694090,95.375461,95.633408,95.112865,...,95.088797,93.818448,94.763476,94.748871,94.360141,94.287522,94.869719,93.555549,94.617034,94.445284
7,88.378440,90.087485,93.297954,90.554074,91.843508,91.977776,95.563786,96.154368,96.358357,95.938360,...,95.878228,94.838885,95.617535,95.601125,95.345353,95.288821,95.746205,94.670708,95.510526,95.358885
8,90.260758,91.706611,94.411442,92.072897,93.238367,93.362649,96.299276,96.779739,96.959679,96.629198,...,96.551900,95.677851,96.353771,96.365783,96.110617,96.113330,96.470780,95.578541,96.242482,96.161477
9,91.816418,93.082039,95.359109,93.408791,94.391981,94.492081,96.921635,97.321604,97.452078,97.178072,...,97.124691,96.417649,96.979108,96.946857,96.763358,96.772814,97.041593,96.305275,96.851376,96.787655


In [ ]:
meowermeow = meowermeow.sort_values(0,axis=0,ascending=False).reset_index(drop=True)

In [ ]:
max(meower.loc[0])

In [ ]:
meowermeow_avg = pd.DataFrame(meower.mean(axis=1))

In [ ]:
meowermeow_avg = meowermeow_avg.reset_index(drop=True).transpose()

In [ ]:
meowermeow_avg.values[0]

In [ ]:
help = blocks.compute().reset_index()

In [ ]:
help

In [ ]:
block_range = (np.arange(1,21))

In [ ]:
plt.bar(block_range,meowermeow_avg.values[0])
plt.title('Model Summary for X-space')
plt.xticks(block_range)
plt.xlabel('Component')
plt.ylabel('Cumulative R\u00b2')

In [ ]:
part_1 = ['r' for i in range(128*6)]
part_2 = ['r' for i in range(180*6)]
part_3 = ['r' for i in range(186*6)]
part_4 = ['r' for i in range(186*6)]
part_5 = ['b' for i in range(360*6)]
part_6 = ['b' for i in range(348*6)]

# part_1 = ['r' for i in range(180*6)]
# part_2 = ['r' for i in range(186*6)]
# part_3 = ['r' for i in range(186*6)]
# part_4 = ['r' for i in range(168*6)]
# part_5 = ['b' for i in range(372*6)]
# part_6 = ['b' for i in range(360*6)]
# part_7 = ['b' for i in range(348*6)]
# part_8 = ['b' for i in range(180*6)]
col = np.concatenate((part_1,part_2,part_3,part_4,part_5,part_6))

#blocks = read_hdfs(root,['201811','201812','201901','201902','201910','201911','201912','202001'])

In [ ]:
plt.scatter(model['t_b'][0][0:680],model['t_b'][1][0:680],c=col[0:680*6],marker='.',alpha=0.25)
plt.title('PC1 vs PC2 Scores Before Retrofit')
plt.xlabel('PC1')
plt.ylabel('PC2')


In [ ]:
len(col)

In [ ]:
plt.scatter(model['t_b'][0][680:8328],model['t_b'][1][680:8328],c=col[680*6:8328],marker='.',alpha=0.25)
plt.title('PC1 vs PC2 Scores After Retrofit')
plt.xlabel('PC1')
plt.ylabel('PC2')

In [ ]:
part_1 = ['r' for i in range(128)]
part_2 = ['r' for i in range(180)]
part_3 = ['r' for i in range(186)]
part_4 = ['r' for i in range(186)]
part_5 = ['b' for i in range(360)]
part_6 = ['b' for i in range(348)]
col = np.concatenate((part_1,part_2,part_3,part_4,part_5,part_6))
# part_1 = ['r' for i in range(180)]
# part_2 = ['r' for i in range(186)]
# part_3 = ['r' for i in range(186)]
# part_4 = ['r' for i in range(168)]
# part_5 = ['b' for i in range(372)]
# part_6 = ['b' for i in range(360)]
# part_7 = ['b' for i in range(348)]
# part_8 = ['b' for i in range(99)]
# col = np.concatenate((part_1,part_2,part_3,part_4,part_5,part_6,part_7,part_8))

In [ ]:
len(col)

In [ ]:


plt.scatter(model['w_T'][0],model['w_T'][1],c=col,marker='.',alpha=0.25)
plt.title('PC1 vs PC2 Super Weights')
plt.xlabel('PC1')
plt.ylabel('PC2')

In [ ]:
super_weights = pd.DataFrame(model['w_T']).transpose()

In [ ]:
super_weights['month'] = col

In [ ]:
import seaborn as sns

In [ ]:
sns.set_theme(style="ticks")
sns.set_palette([sns.color_palette()[0],sns.color_palette()[3]], n_colors=2)
sns.pairplot(super_weights, hue='month',kind='scatter',diag_kind='None',markers='.')


In [ ]:
import scipy.io
from scipy import signal
from scipy.fft import fftshift
import matplotlib.pyplot as plt
mat = scipy.io.loadmat(root+'ambient_20190901_000000.mat')

In [ ]:
data = pd.DataFrame(mat['predat_a']['tdata'][0][0])

In [ ]:
fs = mat['predat_a']['fs'][0][0][0][0]

In [ ]:
fs

In [ ]:
data.iloc[:,0]

In [ ]:
hann_window = signal.windows.hann(2**14)
n_overlap=round(0.66*len(hann_window))

In [ ]:
len(hann_window)

In [ ]:
f, t, Sxx = signal.spectrogram(data.iloc[:,2], fs ,window=hann_window, noverlap=n_overlap)

In [ ]:
plt.pcolormesh(t, f, 10*np.log10(Sxx),shading='gourand')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')